In [ ]:
# This makes a smaller AO2D to work with (just one DF)
import ROOT

# File paths
input_file = "AO2D_o2_ctf_run00544742_orbit0212902048_tf0004599497_epn309.root"

# Open the input ROOT file
infile = ROOT.TFile.Open(input_file, "READ")

# Find the first TDirectory starting with "DF_"
df_dir = None
dir_name = ""
for key in infile.GetListOfKeys():
    name = key.GetName()
    if name.startswith("DF_"):
        # Access the TDirectory
        df_dir = infile.Get(name)
        dir_name = name
        break

if not df_dir:
    raise RuntimeError("No TDirectory starting with 'DF_' found.")

# Open the output file (create if not exist)
output_file = "AO2D_reduced_" + str(dir_name) + ".root"
outfile = ROOT.TFile.Open(output_file, "RECREATE")

# Create the same directory structure in the output file
df_dir_copy = outfile.mkdir(dir_name)

# Move to the newly created directory
df_dir_copy.cd()

# Loop over the keys (trees) inside the "DF_" directory and copy them
for key in df_dir.GetListOfKeys():
    obj = df_dir.Get(key.GetName())
    if isinstance(obj, ROOT.TTree):  # Check if it's a TTree
        # Clone the tree and write it to the corresponding directory in the output file
        obj.CloneTree(-1).Write(key.GetName(), ROOT.TObject.kOverwrite)  # Copy the tree

# Now handle the metaData;1 key (TMap) in the top-level directory
meta_data = infile.Get("metaData")
if meta_data:
    if isinstance(meta_data, ROOT.TMap):
       copied_meta_data = meta_data.Clone()
       outfile.cd()  # Make sure we're at the top-level in the output file
       outfile.WriteObject(meta_data, "metaData")

       # Iterate over the map
       iter = meta_data.MakeIterator()
       entry = iter.Next()
       while entry:
         key = entry
         value = meta_data.GetValue(key)

         # Convert TObjString to Python string
         key_str = key.GetName()
         value_str = value.GetName() if value else "None"
         print(f"{key_str}: {value_str}")
         entry = iter.Next()

        
# Close the files
outfile.Close()
infile.Close()

print(f"Copied all trees from TDirectory '{dir_name}' to '{output_file}'.")

In [ ]:
LHCMaxBunches = 3564;                           # max N bunches
LHCRFFreq = 400.789e6;                          # LHC RF frequency in Hz
LHCBunchSpacingNS = 10 * 1.e9 / LHCRFFreq;      # bunch spacing in ns (10 RFbuckets)
LHCOrbitNS = LHCMaxBunches * LHCBunchSpacingNS; # orbit duration in ns
LHCRevFreq = 1.e9 / LHCOrbitNS;                 # revolution frequency
LHCBunchSpacingMUS = LHCBunchSpacingNS * 1e-3;  # bunch spacing in \mus (10 RFbuckets)
LHCOrbitMUS = LHCOrbitNS * 1e-3;

In [ ]:
from ROOT import o2

run_number = 544742

def retrieve_Aggregated_RunInfos(run_number):
    runInfo = o2.parameters.AggregatedRunInfo.buildAggregatedRunInfo(o2.ccdb.BasicCCDBManager.instance(), run_number)
    detList = o2.detectors.DetID.getNames(runInfo.grpECS.getDetsReadOut())
    assert (run_number == runInfo.runNumber)
    assert (run_number == runInfo.grpECS.getRun())
    return {"SOR" : runInfo.sor,
            "EOR" : runInfo.eor,
            "FirstOrbit" : runInfo.orbitSOR,
            "LastOrbit" : runInfo.orbitEOR,
            "OrbitReset" : runInfo.orbitReset,
            "OrbitsPerTF" : int(runInfo.orbitsPerTF),
            "detList" : detList}

run_info = retrieve_Aggregated_RunInfos(run_number)

# update num of timeframes
# figure out how many timeframes fit into this run range
# take the number of orbits per timeframe and multiply by orbit duration to calculate how many timeframes fit into this run
time_length_inmus = 1000 * (run_info["EOR"] - run_info["SOR"])
ntimeframes = time_length_inmus / (run_info["OrbitsPerTF"] * LHCOrbitMUS)
# also calculate how many orbits fit into the run range
print (f"This run has space for {ntimeframes} timeframes")
    
run_info["ntimeframes"] = ntimeframes

In [ ]:
import uproot
import pandas as pd
import re

In [ ]:
def get_bc_with_timestamps(bc_data, run_info):
  """
  bc_data is a pandas df containing the AO2D basic bunch crossing data
  """
  
  # add a new column to the bc table dynamically
  # this is the time in mu s
  bc_data["timestamp"] = run_info["OrbitReset"] + (bc_data["fGlobalBC"] * LHCBunchSpacingMUS).astype("int64")
  bc_data["timeframeID"] = ((bc_data["fGlobalBC"] - (run_info["FirstOrbit"] * LHCMaxBunches)) / (LHCMaxBunches * run_info["OrbitsPerTF"])).astype("int64")
  bc_data["orbit"] = (bc_data["fGlobalBC"] // LHCMaxBunches).astype("int64")
  bc_data["bc_within_orbit"] = (bc_data["fGlobalBC"] % LHCMaxBunches).astype("int64")
  return bc_data


def get_timeframe_structure(filepath, run_info, max_folders=1, include_dataframe = False, folder_filter=None):
  """
  run_info: The aggregated run_info object for this run
  """
  def find_tree_key(keys, pattern):
     for key in keys:
        key_clean = key
        if re.search(pattern, key_clean, re.IGNORECASE):
            return key_clean
     return None

  file = uproot.open(filepath)
  raw_keys = file.keys()
  #print (raw_keys)
    
  folders = { k.split("/")[0] : 1 for k in raw_keys if "O2bc_001" in k }
  #print (folders)
  folders = [ k for k in folders.keys() ] 
  #print (folders)
  folders = folders[:max_folders]  

  print ("have ", len(raw_keys), f" in file {filepath}")
  # print (folders)

  merged = {} # data containers per file
  for folder in folders:
    if folder_filter != None and folder != folder_filter:
        continue
    #print (f"Looking into {folder}")
    
    # Find correct table names using regex
    bc_key = find_tree_key(raw_keys, f"^{folder}/O2bc_001")
    bc_data = file[bc_key].arrays(library="pd")

    # collision data
    coll_key = find_tree_key(raw_keys, f"^{folder}/O2coll.*_001")
    coll_data = file[coll_key].arrays(library="pd")
    
    # extend the data
    bc_data = get_bc_with_timestamps(bc_data, run_info)
    
    # do the splice with collision data
    bc_data_coll = bc_data.iloc[coll_data["fIndexBCs"]].reset_index(drop=True)
    # this is the combined table containing collision data associated to bc and time information
    combined = pd.concat([bc_data_coll, coll_data], axis = 1)
    
    # do the actual timeframe structure calculation; we only take collisions with a trigger decision attached
    triggered = combined[combined["fTriggerMask"] != 0]
    timeframe_structure = triggered.groupby('timeframeID').apply(
      lambda g: list(zip(g['fGlobalBC'], g['fPosX'], g['fPosY'], g['fPosZ'], g['orbit'], g['bc_within_orbit'], g['fCollisionTime']))
    ).reset_index(name='position_vectors')
    
    folderkey = folder + '@' + filepath
    merged[folderkey] = timeframe_structure # data per folder
    if include_dataframe:
        merged["data"] = combined
  
  # annotate which timeframes are available here and from which file

  return merged

In [ ]:
merged=get_timeframe_structure('AO2D_o2_ctf_run00544742_orbit0212902048_tf0004599497_epn309.root', run_info, max_folders=1000)

# Iterating over rows
for key in merged:
  result = merged[key]
  print (key)
  for index, row in result.iterrows():
    tf = row['timeframeID']
    num_cols = len(row['position_vectors'])
    print (f"Timeframe {tf} has {num_cols} collisions")
    orbit = row['position_vectors'][0][4]
    bc = row['position_vectors'][0][0]
    bc_relative = row['position_vectors'][0][5]
    col_time = row['position_vectors'][0][6]
    print (f"Global BC {bc}; orbit {orbit}; bc_relative {bc_relative}; col time {col_time}")

In [ ]:
def fetch_bccoll_to_localFile(alien_file, local_filename):

  # Open the remote file via AliEn
  infile = ROOT.TFile.Open(alien_file, "READ")
  if not infile or infile.IsZombie():
    raise RuntimeError(f"Failed to open {alien_file}")

  # Output local file
  outfile = ROOT.TFile.Open(local_filename, "RECREATE")

  # List of trees to copy
  trees_to_copy = ["O2bc_001", "O2collision_001"]

  # Loop over top-level keys to find DF_ folders
  for key in infile.GetListOfKeys():
    obj = key.ReadObj()
    if obj.InheritsFrom("TDirectory") and key.GetName().startswith("DF_"):
        df_name = key.GetName()
        df_dir = infile.Get(df_name)
        
        # Create corresponding folder in output file
        out_df_dir = outfile.mkdir(df_name)
        out_df_dir.cd()

        # Copy only specified trees if they exist
        for tree_name in trees_to_copy:
            if df_dir.GetListOfKeys().FindObject(tree_name):
                tree = df_dir.Get(tree_name)
                cloned_tree = tree.CloneTree(-1)  # copy all entries
                cloned_tree.Write(tree_name)

        outfile.cd()  # go back to top-level for next DF_

  # Close files
  outfile.Close()
  infile.Close()

In [ ]:
import ROOT
# Connect to AliEn grid
ROOT.TGrid.Connect("alien://")

#alien_file='alien:///alice/data/2023/LHC23zzm/544742/apass5/2350/o2_ctf_run00544742_orbit0137223104_tf0002234530_epn138/009/AO2D.root'
#local_filename='AO2D_skimmed.root'
#fetch_bccoll_to_localFile(alien_file, local_filename)


In [ ]:
filelist = ['alien:///alice/data/2023/LHC23zzm/544742/apass5/0000/o2_ctf_run00544742_orbit0137377824_tf0002239365_epn262/001/AO2D.root',
'alien:///alice/data/2023/LHC23zzm/544742/apass5/0000/o2_ctf_run00544742_orbit0137377824_tf0002239365_epn262/002/AO2D.root',
'alien:///alice/data/2023/LHC23zzm/544742/apass5/0000/o2_ctf_run00544742_orbit0137377824_tf0002239365_epn262/003/AO2D.root',
'alien:///alice/data/2023/LHC23zzm/544742/apass5/0000/o2_ctf_run00544742_orbit0137377824_tf0002239365_epn262/004/AO2D.root',
'alien:///alice/data/2023/LHC23zzm/544742/apass5/0000/o2_ctf_run00544742_orbit0137377824_tf0002239365_epn262/005/AO2D.root',
'alien:///alice/data/2023/LHC23zzm/544742/apass5/0000/o2_ctf_run00544742_orbit0137377824_tf0002239365_epn262/006/AO2D.root',
'alien:///alice/data/2023/LHC23zzm/544742/apass5/0000/o2_ctf_run00544742_orbit0137377824_tf0002239365_epn262/007/AO2D.root',
'alien:///alice/data/2023/LHC23zzm/544742/apass5/0000/o2_ctf_run00544742_orbit0137377824_tf0002239365_epn262/008/AO2D.root',
'alien:///alice/data/2023/LHC23zzm/544742/apass5/0000/o2_ctf_run00544742_orbit0137377824_tf0002239365_epn262/009/AO2D.root',
'alien:///alice/data/2023/LHC23zzm/544742/apass5/0000/o2_ctf_run00544742_orbit0137377824_tf0002239365_epn262/010/AO2D.root']

In [ ]:
import os
import hashlib
timeframe_data = []
for file in filelist[:4]:
    # calculate filename hash
    hash_name = hashlib.sha1(file.encode()).hexdigest()
    local_filename = os.path.join("./", f"AO2D_{hash_name}.root")
    print (f"Fetching data from {file} to local file {local_filename}")
    if not os.path.exists(local_filename):
      fetch_bccoll_to_localFile(file, local_filename)
    else:
      print ("Using file from local cache")
    
    print (f"Analyzing local file")
    merged=get_timeframe_structure(local_filename, run_info, max_folders=1000)
    print ("Got " + str(len(merged)) + " datasets")
    timeframe_data.append(merged)

In [ ]:
print (len(timeframe_data))

In [ ]:
# print stats and create reverse index from timeframe to DF_ and file
timeframe_to_file={}
for d in timeframe_data:
  for key in d:
    result = d[key]
    print (key)
    for index, row in result.iterrows():
      tf = row['timeframeID']
      num_cols = len(row['position_vectors'])
      print (f"Timeframe {tf} has {num_cols} collisions")
      timeframe_to_file[tf] = key
        
print (timeframe_to_file)

In [ ]:
from ROOT import o2
LHCMaxBunches = 3564
def convert_to_digicontext(aod_timeframe=None, timeframeID=-1):
    """
    converts AOD collision information from AO2D to collision context
    which can be used for MC
    """
    # we create the digitization context object
    digicontext=o2.steer.DigitizationContext()
    
    # we can fill this container
    parts = digicontext.getEventParts()
    # we can fill this container
    records = digicontext.getEventRecords()
    # copy over information
    maxParts = 1
    
    entry = 0
    vertices = ROOT.std.vector("o2::math_utils::Point3D<float>")()
    vertices.resize(len(aod_timeframe))
    
    colindex = 0
    for colindex, col in enumerate(aod_timeframe):
        # we make an event interaction record
        pvector = ROOT.std.vector("o2::steer::EventPart")()
        pvector.push_back(o2.steer.EventPart(0, colindex))
        parts.push_back(pvector)
        
        orbit = col[4]
        bc_within_orbit = col[5]
        interaction_rec = o2.InteractionRecord(bc_within_orbit, orbit)
        col_time_relative_to_bc = col[6] # in NS
        time_interaction_rec = o2.InteractionTimeRecord(interaction_rec, col_time_relative_to_bc)
        records.push_back(time_interaction_rec)
        vertices[colindex].SetX(col[1])
        vertices[colindex].SetY(col[2])
        vertices[colindex].SetZ(col[3])
        
    digicontext.setInteractionVertices(vertices)
    digicontext.setNCollisions(vertices.size())
    digicontext.setMaxNumberParts(maxParts)
    
    # set the bunch filling ---> NEED to fetch it from CCDB
    # digicontext.setBunchFilling(bunchFillings[0]);
    
    prefixes = ROOT.std.vector("std::string")();
    prefixes.push_back("sgn")
    
    digicontext.setSimPrefixes(prefixes);
    digicontext.printCollisionSummary();
    digicontext.saveToFile(f"collission_context_{timeframeID}.root")
    

    
convert_to_digicontext(timeframe_data[0]['DF_2339649212135456'].iloc[0]['position_vectors'], timeframe_data[0]['DF_2339649212135456'].iloc[0]['timeframeID'])

In [ ]:
timeframe_data[0]['DF_2339649212135456'].iloc[0]['position_vectors']

In [ ]:
for d in timeframe_data:
  for key in d:
    result = d[key]
    for index, row in result.iterrows():
      tf = row['timeframeID']
      cols = row['position_vectors']
      convert_to_digicontext(cols, tf)


In [ ]:
print (run_info)

In [ ]:
import os
# define which TIMEFRAME ID to simulate
os.environ["ANCHOR_TIMEFRAME_ID"] = "2239364"
os.environ["ANCHOR_RUN_NUMBER"] = "544742"
os.environ["ANCHOR_FIRST_ORBIT"] = "65718176"


In [ ]:
# produce a MC workflow for the given timestamp and conditions ----> This will produce an MC dataset

In [ ]:
%%bash
# open MC AO2D and contrast to DATA AO2D
export O2DPG_ROOT=/home/swenzel/alisw/O2DPG
pwd
if [ ! -d "MC_${ANCHOR_TIMEFRAME_ID}" ]; then
  mkdir MC_${ANCHOR_TIMEFRAME_ID}
fi
cd MC_${ANCHOR_TIMEFRAME_ID}
pwd

SIMENGINE=TGeant4
NWORKERS=8
${O2DPG_ROOT}/MC/bin/o2dpg_sim_workflow.py -eCM 14000  -col pp -gen pythia8 -proc cdiff -tf 1     \
                                                       -ns 2000 -e ${SIMENGINE}                   \
                                                       -j ${NWORKERS} -interactionRate 500000     \
                                                       -run ${ANCHOR_RUN_NUMBER} -seed 624        \
                                                       -productionTag "alibi_O2DPG_pp_minbias"    \
                                                       --production-offset ${ANCHOR_TIMEFRAME_ID} \
                                                       --data-anchoring ../                       \
                                                       --first-orbit ${ANCHOR_FIRST_ORBIT}

${O2DPG_ROOT}/MC/bin/o2_dpg_workflow_runner.py -f workflow.json -tt aod

In [ ]:
MC_AOD_FILE='MC_2239364/AO2D.root'
mc_data=get_timeframe_structure(MC_AOD_FILE, run_info, max_folders=1, include_dataframe=True)


In [ ]:
mc_data

In [ ]:
timeframe_to_file[2239364]

In [ ]:
data_AOD='AO2D_a93f9e01edd28f0253aecfe2c40bebf0ecc2358a.root'
data_data=get_timeframe_structure(data_AOD, run_info, max_folders=1, include_dataframe=True, folder_filter='DF_2339649212135456')


In [ ]:
data_df=data_data['data'][data_data['data']['timeframeID']==2239364]

In [ ]:
tmp

In [ ]:
mc_df = mc_data['data']

In [ ]:
# filter
# Step 1: Find intersection of X values
common_bc = set(mc_df['fGlobalBC']).intersection(data_df['fGlobalBC'])
len(common_bc)

# Step 2: Filter both DataFrames
df1_filtered = mc_df[mc_df['fGlobalBC'].isin(common_bc)]
df2_filtered = data_df[data_df['fGlobalBC'].isin(common_bc)]
len(df1_filtered)
len(df2_filtered)
# Step 3: Rename columns in df2 (except 'X')
df2_renamed = df2_filtered.rename(columns={col: f"{col}_data" for col in df2_filtered.columns if col != 'fGlobalBC'})
#result = pd.merge(df1_filtered, df2_renamed, on='fGlobalBC', how='inner')
print (df2_renamed)
# Step 3: Concatenate the filtered DataFrames
# embedded_events = df1_filtered.join(df2_renamed).reset_index()
#embedded_events = pd.concat([df1_filtered, df2_filtered], ignore_index=True)

#df2_renamed = df2_filtered.rename(columns={col: f"{col}_df2" for col in df2_filtered.columns if col != 'X'})

# Step 4: Merge the two DataFrames on 'X'
embedded_events = pd.merge(df1_filtered, df2_renamed, on='fGlobalBC', how='inner')

In [ ]:
print (embedded_events)

In [ ]:
embedded_events[['fGlobalBC','orbit','bc_within_orbit','fPosX','fPosX_data', 'fPosY','fPosY_data','fPosZ','fPosZ_data' ]]